In [87]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as st
import yfinance as yf

In [88]:
tickers = ["GOOGL", "AMZN", "TSLA"]

dane = {}
for ticker in tickers:
 dane[ticker] = yf.download(ticker, start="2023-01-01", end="2023-9-21")
data = pd.concat(dane, axis = 1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [89]:
data.head()

GOOGL                                                        \
                 Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2023-01-03  89.589996  91.050003  88.519997  89.120003  89.017838  28131200   
2023-01-04  90.349998  90.650002  87.269997  88.080002  87.979027  34854800   
2023-01-05  87.470001  87.570000  85.900002  86.199997  86.101181  27194400   
2023-01-06  86.790001  87.690002  84.860001  87.339996  87.239868  41381500   
2023-01-09  88.360001  90.050003  87.860001  88.019997  87.919090  29003900   

                 AMZN                                                        \
                 Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2023-01-03  85.459999  86.959999  84.209999  85.820000  85.820000  76706000   
2023-01-04  86.550003  86.980003  83.360001  85.139999  85.139999  68885100   
2023-01-05  85.330002  85.419998  83.070000  83.120003  83.120003  67930800   
2023-01-06  83.029999  86.400002  81.430000  86.080002  86.080002  83303400   
2023-01-09  87.459999  89.480003  87.080002  87.360001  87.360001  65266100   

                  TSLA                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2023-01-03  118.470001  118.800003  104.639999  108.099998  108.099998   
2023-01-04  109.110001  114.589996  107.519997  113.639999  113.639999   
2023-01-05  110.510002  111.750000  107.160004  110.339996  110.339996   
2023-01-06  103.000000  114.389999  101.809998  113.059998  113.059998   
2023-01-09  118.959999  123.519997  117.110001  119.769997  119.769997   

                       
               Volume  
Date                   
2023-01-03  231402800  
2023-01-04  180389000  
2023-01-05  157986300  
2023-01-06  220911100  
2023-01-09  190284000

## ZADANIE 1 (statystyki opisowe)

In [90]:
data.describe()

GOOGL                                                  \
             Open        High         Low       Close   Adj Close   
count  180.000000  180.000000  180.000000  180.000000  180.000000   
mean   112.626611  114.157944  111.500055  112.889945  112.760530   
std     15.300018   15.298348   15.212116   15.169495   15.152105   
min     85.980003   87.570000   84.860001   86.199997   86.101181   
25%     99.677500  101.182503   97.560001  100.187500  100.072647   
50%    115.755001  117.814999  114.880001  116.480000  116.346466   
75%    125.145002  126.517500  123.332499  124.764999  124.621969   
max    138.080002  139.160004  136.619995  138.210007  138.051559   

                           AMZN                                      \
             Volume        Open        High         Low       Close   
count  1.800000e+02  180.000000  180.000000  180.000000  180.000000   
mean   3.425659e+07  114.388723  115.947222  112.947667  114.540000   
std    1.371618e+07   17.208651   17.145985   17.133553   17.054723   
min    1.446790e+07   83.029999   85.419998   81.430000   83.120003   
25%    2.630218e+07   99.039999  100.802500   97.607502   99.152500   
50%    3.054180e+07  111.040001  112.959999  109.285000  111.689999   
75%    3.724945e+07  130.309998  131.532505  128.860001  130.167496   
max    1.194550e+08  145.080002  145.860001  142.949997  144.850006   

                                       TSLA                          \
        Adj Close        Volume        Open        High         Low   
count  180.000000  1.800000e+02  180.000000  180.000000  180.000000   
mean   114.540000  6.163070e+07  208.678612  213.288222  204.416167   
std     17.054723  2.187681e+07   46.100524   46.351062   45.540207   
min     83.120003  2.826480e+07  103.000000  111.750000  101.809998   
25%     99.152500  4.795572e+07  177.865002  183.209999  175.149998   
50%    111.689999  5.674265e+07  199.540001  204.214996  194.364998   
75%    130.167496  6.914592e+07  251.277500  258.982506  246.824997   
max    144.850006  1.581542e+08  296.040009  299.290009  289.519989   

                                             
            Close   Adj Close        Volume  
count  180.000000  180.000000  1.800000e+02  
mean   209.264666  209.264666  1.442161e+08  
std     45.695701   45.695701  3.835935e+07  
min    108.099998  108.099998  8.316600e+07  
25%    180.137501  180.137501  1.128891e+08  
50%    201.010002  201.010002  1.359068e+08  
75%    254.510002  254.510002  1.681930e+08  
max    293.339996  293.339996  3.065906e+08

## ZADANIE 2 (model OLS)

Zrobimy najpierw macierz korelacji:

In [127]:
data2 = data.loc[:, (slice(None), 'Adj Close')]
correlation_matrix = data2.corr()
correlation_matrix

,,GOOGL,AMZN,TSLA
,,Adj Close,Adj Close,Adj Close
GOOGL,Adj Close,1.000000,0.960345,0.766174
AMZN,Adj Close,0.960345,1.000000,0.846925
TSLA,Adj Close,0.766174,0.846925,1.000000


GOOGL i AMZN mają najwyższą korelację (0.960345), co sugeruje, że GOOGL może być dobrą zmienną zależną 
𝑌, a AMZN i TSLA jako zmienne niezależne 𝑋1 i 𝑋2.

Zmienne do modelu OLS:
- $Y$: cena zamknięcia skorygowana o działania korporacyjne (`Adj Close`) dla GOOGLE
- $X_1$: cena zamknięcia skorygowana o działania korporacyjne (`Adj Close`) dla AMAZON
- $X_2$: cena zamknięcia skorygowana o działania korporacyjne (`Adj Close`) dla TESLA

Postaci modelów:

$$Y = \alpha_0 + \alpha_1 X_1 + \alpha_2 X_2 + \varepsilon \\ Y = \beta_0 + \beta_1 X_1 + \varepsilon \\ Y = \gamma_0 + \gamma_1 X_2 + \varepsilon$$

In [102]:
Y = data2['GOOGL']
X1 = data2['AMZN']
X2 = data2['TSLA']

In [105]:
# Model Y ~ X1 + X2
X1_X2 = sm.add_constant(pd.concat([X1, X2], axis=1))  
model1 = sm.OLS(Y, X1_X2).fit()

# Model Y ~ X1
X1_const = sm.add_constant(X1)
model2 = sm.OLS(Y, X1_const).fit()

# Model Y ~ X2
X2_const = sm.add_constant(X2)
model3 = sm.OLS(Y, X2_const).fit()

## ZADANIE 3 (ocena dopasowanych modeli)

In [104]:
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:              Adj Close   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     1178.
Date:                Thu, 20 Jun 2024   Prob (F-statistic):          5.23e-103
Time:                        18:49:51   Log-Likelihood:                -504.67
No. Observations:                 180   AIC:                             1015.
Df Residuals:                     177   BIC:                             1025.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.2319      2.138      5.721      0.0

Współczynnik determinacji (`R-squared`) równy 0.930, co oznacza, że 93% zmienności zmiennej zależnej (GOOGL) jest wyjaśnione przez zmienne niezależne (AMZN i TSLA). Jest to wysoki poziom, co sugeruje, że model jest dobrze dopasowany do danych.

P-value ocenia istotność statystyczną każdego współczynnika. Wszystkie współczynniki w tym modelu mają bardzo niskie P-value (bliskie zeru), co oznacza, że każdy z nich jest istotny statystycznie.

Prob (F-statistic) ocenia ogólną istotność modelu regresji. Niskie wartości (5.23e-103 w tym przypadku) sugerują, że przynajmniej jedna z zmiennych niezależnych jest istotnie związana ze zmienną zależną.

In [107]:
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:              Adj Close   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     2112.
Date:                Thu, 20 Jun 2024   Prob (F-statistic):          1.15e-100
Time:                        18:55:47   Log-Likelihood:                -514.27
No. Observations:                 180   AIC:                             1033.
Df Residuals:                     178   BIC:                             1039.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         15.0339      2.150      6.993      0.0

Współczynnik determinacji (`R-squared`) równy 0.922,co oznacza że 92.2% zmienności zmiennej zależnej (GOOGL) jest wyjaśnione przez zmienną niezależną (AMZN). Jest to bardzo wysoki poziom, co sugeruje, że model dobrze dopasowuje się do danych.

P-value ocenia istotność statystyczną każdego współczynnika. Wszystkie współczynniki w tym modelu mają bardzo niskie P-value (bliskie zeru), co oznacza, że każdy z nich jest istotny statystycznie.

Prob (F-statistic) ocenia ogólną istotność modelu regresji. Niskie wartości (1.15e-100 w tym przypadku) sugerują, że zmienna niezależna (AMZN) jest istotnie związana ze zmienną zależną.

In [109]:
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:              Adj Close   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     253.0
Date:                Thu, 20 Jun 2024   Prob (F-statistic):           5.10e-36
Time:                        18:58:57   Log-Likelihood:                -664.58
No. Observations:                 180   AIC:                             1333.
Df Residuals:                     178   BIC:                             1340.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         59.5961      3.421     17.423      0.0

Współczynnik determinacji (`R-squared`) równy 0.587, co oznacza, że 58.7% zmienności zmiennej zależnej (GOOGL) jest wyjaśnione przez zmienną niezależną (TSLA). Jest to niższy poziom w porównaniu do poprzednich modeli, co sugeruje, że TSLA może słabiej tłumaczyć zmienność GOOGL niż AMZN.

P-value ocenia istotność statystyczną każdego współczynnika. Wszystkie współczynniki w tym modelu mają bardzo niskie P-value (bliskie zeru), co oznacza, że każdy z nich jest istotny statystycznie

Prob (F-statistic) ocenia ogólną istotność modelu regresji. Niskie wartości (5.10e-36 w tym przypadku) sugerują, że zmienna niezależna (TSLA) jest istotnie związana ze zmienną zależną.

Model Y ~ X2 (gdzie Y to GOOGL, a X2 to TSLA) wykazuje niższy poziom dopasowania w porównaniu do modeli wcześniejszych (Y ~ X1 + X2 i Y ~ X1). Istotność statystyczna współczynnika TSLA sugeruje, że mimo niższego wspólczynnika determinacji (`R-squared`), zmienna TSLA jest istotnym predyktorem GOOGL. Jednak występują problemy diagnostyczne, takie jak silna autokorelacja i niestandardowy rozkład reszt, które mogą wskazywać na potencjalne problemy w interpretacji wyników.

## ZADANIE 4 (weryfikacja hipotez)


### a) Test dla średnich



#### Założenia:
##### Normalność
   

 $H_0:$ Zmienne $X1$ i $X2$ pochodzą z populacji o rozkładzie normalnym \
 $H_1:$ Zmienne $X1$ i $X2$ pochodzą z populacji o rozkładzie innym, niż normalny 

In [120]:
shapiro_X1 = st.shapiro(X1)
shapiro_X2 = st.shapiro(X2)
print("Test Shapiro-Wilka dla X1:")
print("Statystyka testowa:", shapiro_X1.statistic)
print("Wartość p:", shapiro_X1.pvalue)
print("\nTest Shapiro-Wilka dla X2:")
print("Statystyka testowa:", shapiro_X2.statistic)
print("Wartość p:", shapiro_X2.pvalue)

Test Shapiro-Wilka dla X1:
Statystyka testowa: 0.913341760635376
Wartość p: 8.116945338088044e-09

Test Shapiro-Wilka dla X2:
Statystyka testowa: 0.9583266973495483
Wartość p: 3.5559252864914015e-05


P-wartość jest mniejsza niż ustalony poziom istotności 0.05, więc odrzucamy hipotezę zerową na korzyść alternatywnej. Możemy przypuściać że zmienne $X1$ i $X2$ pochodzą z populacji o rozkładzie innym, niż normalny.

##### Równość wariancji

$H_0: \sigma_{x1}^2 = \sigma_{x2}^2$ \
$H_1: \sigma_{x1}^2 \neq \sigma_{x2}^2$ 

 

In [134]:
X1_values = data2['AMZN'].to_numpy()  
X2_values = data2['TSLA'].to_numpy()  

# Usunięcie wartości NaN
X1_values = X1_values[~np.isnan(X1_values)]
X2_values = X2_values[~np.isnan(X2_values)]

# Test Levene'a dla równości wariancji
levene_test = st.levene(X1_values, X2_values)
print("\nTest Levene'a dla równości wariancji:")
print("Statystyka:", levene_test.statistic)
print("Wartość p:", levene_test.pvalue)


Test Levene'a dla równości wariancji:
Statystyka: 130.01854336496808
Wartość p: 6.662077310908193e-26


P-wartość jest mniejsza niż ustalony poziom istotności 0.05, więc odrzucamy hipotezę zerową na korzyść alternatywnej.Możemy przypuściać że zmienne $X_1$ i $X_2$ mają różne wariancje.


#### Test

Na podstawie sprawdzonych założeń możemy użyć T-test, który będzie odpowiadał założeniam.

$$H_0: \mu_1 = \mu_2 \\ H_1: \mu_1 \neq \mu_2$$

In [138]:
t_stat, t_pvalue = st.ttest_ind(X1, X2, equal_var=False)
print("\nNieparametryczny T-test dla równości średnich:")
print("Wartość t:", t_stat)
print("Wartość p:", t_pvalue)


Nieparametryczny T-test dla równości średnich:
Wartość t: [-26.05587756]
Wartość p: [2.7432846e-70]


P-wartość jest mniejsza niż ustalony poziom istotności 0.05, więc odrzucamy hipotezę zerową na korzyść alternatywnej.Możemy przypuściać że średnie są różne.

### b) Test zgodności (KS test)

$$H_0: F_1 = F_2 \\ H_1: F_1 \neq F_2$$

In [141]:
Y_values = Y.to_numpy()
Y_values = Y_values[~np.isnan(Y_values)]

ks_stat, ks_pvalue = st.ks_2samp(Y_values, X1_values)
print("\nTest Kolmogorova-Smirnova dla równości rozkładów (Y vs X1):")
print("Statystyka KS:", ks_stat)
print("Wartość p:", ks_pvalue)


Test Kolmogorova-Smirnova dla równości rozkładów (Y vs X1):
Statystyka KS: 0.16111111111111112
Wartość p: 0.018566878576656717


P-wartość jest mniejsza niż ustalony poziom istotności 0.05, więc odrzucamy hipotezę zerową na korzyść alternatywnej.Możemy przypuściać,że zmienne $Y$ i $X_1$ mają różne rozkłady.